In [1]:
from kaggle_secrets import UserSecretsClient

from huggingface_hub import login

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("hf")
login(token=secret_value_0)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
!pip install --upgrade datasets
!pip install --upgrade accelerator


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 3.1 MB/s eta 0:00:00


In [3]:
!pip install --upgrade numpy==1.26.4

In [4]:
!pip install transformers torch pandas scikit-learn tqdm datasets

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TrainingArguments, AutoModelForSequenceClassification

# Load dataset
data = pd.read_csv("/kaggle/input/orientationtr/orientation-tr-train.tsv", sep="\t")

# Preprocessing: Handle missing values
data = data.dropna(subset=[ 'text_en'])

# Stratified train-test split
train_data, val_data = train_test_split(
    data, test_size=0.1, stratify=data["label"], random_state=42
)

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

def tokenize_data(texts, labels):
    return tokenizer(
        list(texts), padding=True, truncation=True, return_tensors="pt"
    ), labels

train_tokens, train_labels = tokenize_data(train_data["text_en"], train_data["label"])
val_tokens, val_labels = tokenize_data(val_data["text_en"], val_data["label"])

model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2, torch_dtype="auto")



tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00


In [7]:
# Check class distribution
label_counts = data["label"].value_counts()

# Print class distribution
print("Class Distribution:")
print(label_counts)

# Calculate imbalance ratio
total_samples = label_counts.sum()
imbalance_ratios = label_counts / total_samples
print("\nClass Imbalance Ratios:")
print(imbalance_ratios)


Class Distribution:
label
1    9390
0    6748
Name: count, dtype: int64

Class Imbalance Ratios:
label
1    0.581856
0    0.418144
Name: count, dtype: float64


In [8]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import TrainingArguments, Trainer


In [9]:

from datasets import Dataset

# Combine the tokenized data into datasets
train_dataset = Dataset.from_dict({
    "input_ids": train_tokens["input_ids"],
    "attention_mask": train_tokens["attention_mask"],
    "labels": train_labels,
})

eval_dataset = Dataset.from_dict({
    "input_ids": val_tokens["input_ids"],
    "attention_mask": val_tokens["attention_mask"],
    "labels": val_labels,
})

training_args = TrainingArguments(
    output_dir="./orientation-tr",
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=50,
    load_best_model_at_end=True,
    report_to="none"
)
# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)


trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.392000,0.332477,0.854399
2,0.275300,0.326708,0.869888
3,0.179700,0.324783,0.867410


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=1362, training_loss=0.32105192695245166, metrics={'train_runtime': 2525.6209, 'train_samples_per_second': 17.252, 'train_steps_per_second': 0.539, 'total_flos': 1.146427490414592e+16, 'train_loss': 0.32105192695245166, 'epoch': 3.0})

In [10]:
from kaggle_secrets import UserSecretsClient

from huggingface_hub import login

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("hf")
login(token=secret_value_0)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [11]:
# Load dataset
import pandas as pd


model_id = "meta-llama/Llama-3.2-1B"
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model=model_id, device_map="auto")

# Define candidate labels
candidate_labels = ["left wing", "right wing"]

# Predict labels using the classifier
predictions = []
i = 0

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [30]:
data = pd.read_csv("/kaggle/input/orientationtr/orientation-tr-train.tsv", sep="\t")

# Preprocessing: Handle missing values
data = data.dropna(subset=[ 'text_en'])
predictions = []
train_data, val_data = train_test_split(
    data, test_size=0.1, stratify=data["label"], random_state=42
)

In [25]:
print(len(val_data["label"]))

print(len(predictions_mapped))

807
13693


In [31]:
i = 0
predictions = []
for text in val_data["text_en"]:
    result = classifier(text, candidate_labels=candidate_labels)
    # Get the label with the highest score
    if 'labels' in result:
      predictions.append(result['labels'][0])
    else:
        print(f"Unexpected result format: {result}")
        predictions.append(None)  # Or handle appropriately
        print(i)
    if (i == 10):
        print(i)
        i=0
        
    i += 1

# Map predictions and true labels to a common format
# Assuming `data['label']` has values like 0 for "right wing" and 1 for "left wing"
true_labels = val_data["label"].map({1: "right wing", 0: "left wing"})
 
from sklearn.metrics import accuracy_score

# Map the predicted labels back to 0 and 1
label_mapping = {"left wing": 0, "right wing": 1}
predictions_mapped = [label_mapping[label] for label in predictions]

# Calculate accuracy
accuracy = accuracy_score(val_data["label"], predictions_mapped)
print(f"Accuracy: {accuracy:.4f}")



10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
Accuracy: 0.4287


FİRST TASK İS DONE